In [1]:
from deap import base, creator, tools
import random
import itertools

# define the signal and meaning spaces
signals = ['a', 'b', 'c', 'd']  # All possible signals
meanings = [0, 1, 2, 3]         # All possible meanings

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax, local_state=None)

toolbox = base.Toolbox()

toolbox.register("attr_signal", random.choice, signals)
toolbox.register("attr_meaning", random.choice, meanings)
toolbox.register("attr_local_state", random.randint, 0, 3)
toolbox.register("attr_global_state", random.randint, 0, 3)

GLOBAL = toolbox.attr_global_state()
print("Global State: ", GLOBAL)

def create_individual():
    trans_genes = [toolbox.attr_signal() for _ in meanings]  
    recept_genes = [toolbox.attr_meaning() for _ in signals]  
    individual = creator.Individual(trans_genes + recept_genes)
    individual.local_state = toolbox.attr_local_state() 
    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


GLOBAL_STATE = random.randint(0, 3)

def pairwise_communication(speaker, listener):

    # speaker encodes signals
    local_signal = speaker.local_state
    global_signal = GLOBAL

    semantic_local = speaker[local_signal]
    semantic_global = speaker[global_signal]

    # listener decodes local signal
    if semantic_local == 'a':
        decoded_local = listener[4]
    elif semantic_local == 'b':
        decoded_local = listener[5]
    elif semantic_local == 'c':
        decoded_local = listener[6]
    else:
        decoded_local = listener[7]

    # listener decodes global signal
    if semantic_global == 'a':
        decoded_global = listener[4]
    elif semantic_global == 'b':
        decoded_global = listener[5]
    elif semantic_global == 'c':
        decoded_global = listener[6]
    else:
        decoded_global = listener[7]

    fitness = 0
    if local_signal == decoded_local:
        fitness += 1
    if global_signal == decoded_global:
        fitness += 1
    return fitness   

# evaluate group
def evaluate_group(group):
    for ind in group:
        ind.fitness.values = (0,)  # reset fitness
    
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

# create pop
population = toolbox.population(n=100)

GROUP_SIZE = 5

# eval groups
for i in range(0, len(population), GROUP_SIZE):
    group = population[i:i + GROUP_SIZE]
    evaluate_group(group)

for ind in population:
    print(ind, ind.fitness.values)


Global State:  3
['d', 'a', 'b', 'd', 0, 3, 1, 3] (4.0,)
['b', 'c', 'c', 'a', 2, 1, 2, 1] (5.0,)
['d', 'a', 'a', 'b', 3, 0, 1, 2] (5.0,)
['a', 'b', 'a', 'c', 1, 0, 3, 3] (5.0,)
['a', 'a', 'c', 'c', 3, 3, 1, 2] (5.0,)
['c', 'c', 'a', 'a', 2, 2, 1, 1] (4.0,)
['b', 'b', 'c', 'd', 3, 0, 3, 0] (5.0,)
['c', 'b', 'a', 'c', 0, 1, 1, 1] (4.0,)
['d', 'a', 'd', 'b', 0, 2, 2, 3] (3.0,)
['c', 'b', 'a', 'a', 3, 3, 3, 2] (6.0,)
['a', 'd', 'd', 'c', 1, 0, 3, 0] (1.0,)
['b', 'c', 'c', 'a', 3, 3, 2, 1] (6.0,)
['d', 'a', 'a', 'b', 3, 1, 1, 0] (6.0,)
['c', 'a', 'b', 'b', 1, 2, 2, 3] (4.0,)
['d', 'd', 'b', 'a', 1, 3, 2, 3] (7.0,)
['b', 'c', 'd', 'b', 3, 2, 3, 1] (9.0,)
['a', 'd', 'b', 'a', 2, 2, 2, 3] (4.0,)
['c', 'a', 'd', 'd', 0, 2, 2, 2] (5.0,)
['a', 'c', 'd', 'c', 1, 0, 3, 3] (6.0,)
['b', 'd', 'b', 'c', 1, 3, 2, 2] (6.0,)
['b', 'c', 'b', 'b', 0, 0, 2, 1] (3.0,)
['c', 'c', 'a', 'd', 3, 0, 2, 0] (4.0,)
['b', 'c', 'b', 'a', 2, 3, 0, 2] (7.0,)
['c', 'd', 'a', 'a', 1, 1, 2, 1] (3.0,)
['d', 'b', 'd', 'b', 2,